In [1]:
import pandas as pd
import numpy as np
import os
import json
import datetime
from tqdm import tqdm
import matplotlib.pyplot as plt
import statsmodels.api as sm
import string
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
import random
from sklearn.feature_extraction import text
import nltk
from nltk.corpus import stopwords
from string import punctuation
from nltk import pos_tag
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

In [2]:
folder_path = "../ECE219_tweet_data/"
files= os.listdir(folder_path)
print(files)

['tweets_#nfl.txt', 'tweets_#superbowl.txt', 'tweets_#sb49.txt', 'tweets_#patriots.txt', 'tweets_#gohawks.txt', 'tweets_#gopatriots.txt']


In [3]:
data={}
a = []
for file_ in files:
    location=[]
    text=[]

    key = file_[7:-4]
    with open(folder_path + file_, encoding="utf8") as f:
        print("Starting for",key)
        for line in tqdm(f):
            tweet = json.loads(line,strict=False)
            location.append(tweet['tweet']['user']['location'])
            text.append(tweet['tweet']['text'])
    
    data = {'location':location,
                'text':text}
    print("done for",key)
    a.append(pd.DataFrame.from_dict(data))       
dfs = pd.concat(a)

1288it [00:00, 12872.43it/s]

Starting for #nfl


233022it [00:15, 15432.17it/s]
1269it [00:00, 12688.71it/s]

done for #nfl
Starting for #superbowl


1213813it [01:14, 16390.85it/s]


...

done for #patriots
Starting for #gohawks


169122it [00:10, 16135.56it/s]
1498it [00:00, 14977.44it/s]

done for #gohawks
Starting for #gopatriots


23511it [00:01, 15871.17it/s]


done for #gopatriots


In [4]:
dfs

,location,text
0,,Who do you have?!?! #nfl #NFLPlayoffs #Packers...
1,Fine Art America,http://t.co/H5JADypiEB #billbelichick #NFL #NF...
2,"Seattle, WA",One more week until the #Seahawks begin the #N...
3,"Virginia Beach, Va",We have NFLSHOP on our site! 3% cash back and ...
4,"Cheshire, CT","Most @SuperBowl wins: #Steelers (6), #49ers &a..."
5,"Cheshire, CT","Most @SuperBowl losses: #Broncos (5), #Patriot..."
6,"Seattle, WA",@Seahawks Sonics fans are in! #ImIn #NFLPlayo...
7,,#gohawks #12thman #nfl #ninernation #49ers #go...
8,,Who is the better QB?\n\nRT- Joe Montana\n\nFA...
9,Seattle,Reppin' for my #Seahawks 🏈 #NFLPlayoffs #GoHaw...


In [5]:
mass=set(["boston","bellevue","masachusettes","mass","ma","springfield"])
mass

{'bellevue', 'boston', 'ma', 'masachusettes', 'mass', 'springfield'}

In [6]:
washington=set(["seattle","washington","wa","kirkland","spokane","redmond"])
washington

{'kirkland', 'redmond', 'seattle', 'spokane', 'wa', 'washington'}

In [7]:
table = str.maketrans(dict.fromkeys(string.punctuation))

In [8]:
dfs['location']=dfs['location'].apply(lambda x: x.lower())
dfs['location']=dfs['location'].apply(lambda x: x.translate(table))

In [9]:
dfs

,location,text
0,,Who do you have?!?! #nfl #NFLPlayoffs #Packers...
1,fine art america,http://t.co/H5JADypiEB #billbelichick #NFL #NF...
2,seattle wa,One more week until the #Seahawks begin the #N...
3,virginia beach va,We have NFLSHOP on our site! 3% cash back and ...
4,cheshire ct,"Most @SuperBowl wins: #Steelers (6), #49ers &a..."
5,cheshire ct,"Most @SuperBowl losses: #Broncos (5), #Patriot..."
6,seattle wa,@Seahawks Sonics fans are in! #ImIn #NFLPlayo...
7,,#gohawks #12thman #nfl #ninernation #49ers #go...
8,,Who is the better QB?\n\nRT- Joe Montana\n\nFA...
9,seattle,Reppin' for my #Seahawks 🏈 #NFLPlayoffs #GoHaw...


In [10]:
dfs['text']=dfs['text'].apply(lambda x: x.lower())

In [11]:
dfs

,location,text
0,,who do you have?!?! #nfl #nflplayoffs #packers...
1,fine art america,http://t.co/h5jadypieb #billbelichick #nfl #nf...
2,seattle wa,one more week until the #seahawks begin the #n...
3,virginia beach va,we have nflshop on our site! 3% cash back and ...
4,cheshire ct,"most @superbowl wins: #steelers (6), #49ers &a..."
5,cheshire ct,"most @superbowl losses: #broncos (5), #patriot..."
6,seattle wa,@seahawks sonics fans are in! #imin #nflplayo...
7,,#gohawks #12thman #nfl #ninernation #49ers #go...
8,,who is the better qb?\n\nrt- joe montana\n\nfa...
9,seattle,reppin' for my #seahawks 🏈 #nflplayoffs #gohaw...


In [12]:
dfs['text'].str.contains('#superbowl')

0        False
1        False
2        False
3        False
4        False
5        False
6        False
7        False
8        False
9        False
10       False
11       False
12       False
13       False
14       False
15       False
16       False
17       False
18       False
19       False
20       False
21       False
22       False
23       False
24       False
25       False
26       False
27       False
28       False
29       False
         ...  
23481     True
23482    False
23483     True
23484     True
23485    False
23486     True
23487     True
23488     True
23489     True
23490     True
23491    False
23492     True
23493    False
23494     True
23495     True
23496    False
23497     True
23498     True
23499     True
23500    False
23501    False
23502    False
23503    False
23504    False
23505     True
23506     True
23507     True
23508    False
23509     True
23510    False
Name: text, Length: 2823738, dtype: bool

In [13]:
dfs=dfs[dfs['text'].str.contains('#superbowl')]

In [14]:
dfs

,location,text
34,,já somos mais de 80 confirmados para assistir ...
36,,"randall cunningham collection, vintage jerseys..."
54,washington,interview with the seattle seahawks superbowl ...
59,souf london,just recalled that #superbowlxlix is next mont...
193,holsworthy sydney australia,#conferencechampionships #nflplayoffs #gbvssea...
213,niagara fallson,@softstarshoes is giving away free moccasins i...
246,,@softstarshoes is giving away free moccasins i...
252,,@softstarshoes is giving away free moccasins i...
255,,check out #larrybird #jeromebettis #baseball #...
265,los angeles,@softstarshoes is giving away free moccasins i...


In [15]:
dfs.drop_duplicates(inplace=True)
len(dfs)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


1199402

In [16]:
dfs['location'].replace('', np.nan, inplace=True)

/usr/local/lib/python3.7/site-packages/pandas/core/generic.py:6586: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [17]:
dfs=dfs.dropna(axis=0)

In [18]:
dfs

,location,text
54,washington,interview with the seattle seahawks superbowl ...
59,souf london,just recalled that #superbowlxlix is next mont...
193,holsworthy sydney australia,#conferencechampionships #nflplayoffs #gbvssea...
213,niagara fallson,@softstarshoes is giving away free moccasins i...
265,los angeles,@softstarshoes is giving away free moccasins i...
280,tokyo japan,i'd take a day off on feb 2 if the #patriots a...
336,london leeds southampton,watch the #superbowlxlix on 1st feb on our big...
396,dogtown,the 2015 super bowl is here! get tickets now a...
413,arizona,reebok nfl super bowl xl detroit 2006 adjustab...
419,washington,interview with the seattle seahawks superbowl ...


In [19]:
dfs = dfs.reset_index(drop=True)

In [20]:
dfs

,location,text
0,washington,interview with the seattle seahawks superbowl ...
1,souf london,just recalled that #superbowlxlix is next mont...
2,holsworthy sydney australia,#conferencechampionships #nflplayoffs #gbvssea...
3,niagara fallson,@softstarshoes is giving away free moccasins i...
4,los angeles,@softstarshoes is giving away free moccasins i...
5,tokyo japan,i'd take a day off on feb 2 if the #patriots a...
6,london leeds southampton,watch the #superbowlxlix on 1st feb on our big...
7,dogtown,the 2015 super bowl is here! get tickets now a...
8,arizona,reebok nfl super bowl xl detroit 2006 adjustab...
9,washington,interview with the seattle seahawks superbowl ...


In [21]:
dfs['label'] = 0
dfs

,location,text,label
0,washington,interview with the seattle seahawks superbowl ...,0
1,souf london,just recalled that #superbowlxlix is next mont...,0
2,holsworthy sydney australia,#conferencechampionships #nflplayoffs #gbvssea...,0
3,niagara fallson,@softstarshoes is giving away free moccasins i...,0
4,los angeles,@softstarshoes is giving away free moccasins i...,0
5,tokyo japan,i'd take a day off on feb 2 if the #patriots a...,0
6,london leeds southampton,watch the #superbowlxlix on 1st feb on our big...,0
7,dogtown,the 2015 super bowl is here! get tickets now a...,0
8,arizona,reebok nfl super bowl xl detroit 2006 adjustab...,0
9,washington,interview with the seattle seahawks superbowl ...,0


In [22]:
idx=[]
for row in dfs.iterrows():
    inter_wash=set(row[1]['location'].split(" ")).intersection(washington)
    inter_mass=set(row[1]['location'].split(" ")).intersection(mass)
    #print(len(inter_wash))
    #print(len(inter_mass))
    if len(inter_mass)==0 and len(inter_wash)==0:
        idx.append(row[0])
    elif len(inter_mass)>0:
        print(row[0])
        dfs['label'][row[0]]=0      #massachusette=0; washington=1
    else:
        dfs['label'][row[0]]=1
        
        
        
        
     

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


76
236


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


303
370
382
431
458
572
605
785
890
1109
1130
1192
1351
1358
1383
1430
1439
1442
1447
1449
1451
1473
1498
1514
1522
1527
1536
1561
1612
1635
1636
1645
1733
1787
1845
1855
1862
1863
1881
1915
1918
1921
1950
1952
1975
1978
1989
2134
2170
2185
2208
2229
2250
2253
2266
2284
2336
2344
2384
2600
2661
2697
2715
2780
2782
2836
2849
2960
2963
2969
2970
3015
3043
3097
3160
3182
3213
3220
3298
3301
3317
3332
3338
3340
3367
3378
3404
3410
3425
3433
3436
3720
3760
3777
3786
3790
3937
3956
3967
3985
3986
4010
4011
4032
4044
4100
4214
4338
4346
4347
4363
4385
4392
4399
4400
4401
4413
4422
4424
4451
4482
4655
4656
4720
4722
4747
4820
4826
4838
4889
4892
4893
4920
4929
4938
4956
4975
5006
5054
5085
5091
5092
5093
5123
5143
5197
5199
5235
5272
5307
5331
5338
5360
5374
5430
5432
5433
5450
5506
5531
5545
5549
5632
5682
5712
5719
5752
5757
5768
5769
5780
5835
5847
5892
5903
5906
5922
5927
5998
6031
6032
6045
6069
6133
6159
6198
6201
6304
6311
6313
6376
6386
6406
6412
6432
6473
6542
6578
6647
6662
6668
6692

53184
53191
53202
53213
53220
53250
53254
53255
53283
53308
53320
53322
53329
53331
53335
53339
53340
53346
53351
53360
53366
53368
53370
53379
53382
53389
53393
53395
53398
53426
53455
53467
53469
53478
53487
53494
53510
53514
53519
53559
53562
53595
53598
53623
53625
53635
53645
53662
53678
53681
53710
53718
53732
53737
53763
53782
53799
53803
53820
53833
53836
53849
53862
53884
53905
53911
53927
53945
53950
53986
53994
53995
54003
54007
54018
54037
54047
54054
54057
54064
54067
54075
54076
54092
54104
54124
54125
54151
54161
54163
54166
54169
54187
54209
54211
54213
54225
54239
54246
54299
54300
54337
54348
54367
54384
54392
54401
54426
54429
54433
54437
54439
54441
54459
54463
54470
54484
54498
54504
54512
54515
54517
54527
54533
54536
54576
54606
54608
54617
54630
54638
54641
54642
54657
54665
54680
54682
54729
54734
54743
54752
54777
54809
54821
54872
54875
54902
54904
54950
54957
54963
54986
54988
54996
55006
55015
55021
55073
55078
55087
55092
55093
55097
55101
55110
55114
5512

...

647842
647969
647978
648016
648059
648081
648267
648355
648371
648466
648471
648528
648556
648577
648849
648969
649011
649016
649197
649311
649516
649597
649610
649731
649752
649796
649949
650079
650114
650129
650187
650296
650310
650337
650365
650497
650654
650665
650672
650817
650927
651343
651356
651401
651459
651500
651516
651548
651603
651655
651779
651850
651901
651927
651995
652057
652065
652105
652336
652423
652516
652561
652612
652651
652743
652759
652813
652839
652910
652932
653046
653052
653069
653097
653116
653128
653330
653339
653345
653377
653402
653411
653426
653452
653558
653935
653982
654078
654225
654264
654293
654314
654459
654537
654574
654580
654585
654619
654698
654793
654837
654845
654962
654986
655018
655059
655168
655240
655305
655468
655476
655760
655881
655905
655914
656057
656168
656199
656231
656430
656445
656493
656498
656807
656864
656879
656933
657066
657078
657131
657164
657170
657444
657460
657522
657532
657580
657630
657682
657684
657725
657883
657886

716959
717022
717036
717058
717206
717229
717277
717313
717439
717453
717510
717546
717581
717604
717724
717757
717795
717806
717809
717817
717934
718070
718133
718197
718262
718325
718343
718382
718435
718441
718479
718495
718549
718716
718724
718730
718779
718846
718927
718936
718969
718993
719058
719134
719196
719199
719221
719272
719383
719463
719491
719551
719558
719559
719571
719631
719632
719640
719667
719668
719773
719775
719848
720052
720102
720150
720178
720225
720270
720301
720347
720359
720407
720413
720484
720510
720574
720595
720714
720726
720766
720844
720851
720901
720984
721023
721104
721138
721145
721189
721255
721390
721420
721489
721494
721501
721524
721590
721598
721619
721689
721691
721795
721812
721833
721871
721893
721960
722016
722088
722099
722137
722216
722280
722327
722335
722359
722411
722488
722665
722824
722848
722855
722884
722916
722936
722959
722964
722988
722999
723011
723026
723030
723085
723111
723166
723167
723173
723230
723275
723285
723338
723360

782245
782260
782265
782396
782425
782441
782485
782536
782590
782615
782623
782627
782648
782657
782738
782804
782807
782939
782976
782994
783047
783088
783093
783108
783158
783199
783208
783212
783323
783334
783360
783369
783373
783422
783436
783464
783519
783535
783572
783646
783651
783663
783680
783693
783742
783757
783764
783782
783806
783808
783819
783851
783904
783948
783961
784022
784066
784087
784178
784199
784234
784245
784257
784281
784358
784417
784434
784527
784529
784547
784598
784606
784613
784698
784711
784731
784747
784752
784755
784890
784908
784913
784921
784925
785029
785127
785131
785161
785247
785249
785266
785351
785397
785438
785487
785510
785542
785564
785581
785591
785606
785627
785688
785751
785853
785872
785903
785926
785951
785975
785996
786009
786053
786058
786075
786093
786120
786135
786159
786175
786250
786255
786276
786343
786350
786410
786427
786430
786514
786536
786583
786589
786674
786705
786706
786758
786763
786904
787066
787068
787119
787137
787146

819713
819742
819830
819844
819852
819873
819874
819880
819882
819909
819917
819929
819933
819998
820024
820051
820055
820069
820070
820205
820210
820216
820233
820252
820333
820369
820380
820383
820405
820428
820487
820503
820554
820595
820609
820620
820639
820642
820650
820723
820726
820730
820740
820745
820760
820793
820826
820846
820876
820901
820913
820955
820998
821074
821102
821167
821176
821204
821251
821259
821265
821271
821295
821345
821354
821368
821373
821382
821419
821425
821435
821489
821504
821517
821529
821557
821579
821585
821595
821599
821615
821619
821644
821655
821661
821664
821696
821726
821746
821748
821765
821787
821789
821790
821801
821802
821830
821838
821864
821867
821906
821923
821929
821976
821978
821979
822003
822010
822065
822068
822073
822077
822082
822084
822092
822095
822097
822098
822141
822143
822150
822157
822167
822239
822245
822252
822253
822295
822297
822331
822337
822342
822343
822375
822376
822383
822400
822403
822410
822418
822445
822447
822455

848435
848454
848461
848464
848481
848512
848532
848550
848604
848677
848709
848710
848750
848757
848808
848813
848818
848822
848849
848863
848935
848966
848991
849003
849087
849143
849187
849271
849273
849278
849298
849357
849403
849407
849430
849467
849512
849595
849644
849683
849690
849702
849706
849756
849771
849800
849823
849905
849920
850027
850090
850177
850261
850281
850302
850312
850429
850523
850577
850599
850649
850664
850732
850883
850905
850917
850957
851033
851056
851238
851375
851377
851435
851525
851594
851611
851620
851651
851685
851690
851751
851766
851773
851967
851980
851986
852020
852027
852108
852169
852178
852242
852249
852332
852491
852510
852519
852548
852601
852606
852642
852677
852682
852696
852707
852746
852813
852832
852882
852885
852886
852891
852900
852901
852902
852910
852940
853007
853021
853107
853137
853155
853163
853182
853188
853204
853211
853217
853219
853231
853240
853246
853270
853273
853301
853317
853320
853332
853339
853343
853345
853349
853370

In [23]:
print(len(idx))

839092


In [24]:
dfs=dfs.drop(idx, axis=0)
dfs

,location,text,label
0,washington,interview with the seattle seahawks superbowl ...,1
9,washington,interview with the seattle seahawks superbowl ...,1
30,in the moment washington,completely ok with being called an insufferabl...,1
40,seattle las vegas,in #honor of #seattle #seahawks last #nfl #pla...,1
70,washington state,#nfl #nflplayoffs #superbowl49 #superbowlchamp...,1
76,springfield ma,can't we just kick this thing off now? this #p...,0
107,seattle wa,@softstarshoes is giving away free moccasins i...,1
122,washington,bam bam kam!! part of the #lob #nflplayoffs #s...,1
130,wa state,@softstarshoes is giving away free moccasins i...,1
207,tacoma wa,you know what time it is. seahawks cake....\n\...,1


In [25]:
dfs

,location,text,label
0,washington,interview with the seattle seahawks superbowl ...,1
9,washington,interview with the seattle seahawks superbowl ...,1
30,in the moment washington,completely ok with being called an insufferabl...,1
40,seattle las vegas,in #honor of #seattle #seahawks last #nfl #pla...,1
70,washington state,#nfl #nflplayoffs #superbowl49 #superbowlchamp...,1
76,springfield ma,can't we just kick this thing off now? this #p...,0
107,seattle wa,@softstarshoes is giving away free moccasins i...,1
122,washington,bam bam kam!! part of the #lob #nflplayoffs #s...,1
130,wa state,@softstarshoes is giving away free moccasins i...,1
207,tacoma wa,you know what time it is. seahawks cake....\n\...,1


In [26]:
y_true=dfs['label']
dfs = dfs.drop(['label'],axis=1)
dfs = dfs.drop(['location'], axis = 1)

In [27]:
dfs

,text
0,interview with the seattle seahawks superbowl ...
9,interview with the seattle seahawks superbowl ...
30,completely ok with being called an insufferabl...
40,in #honor of #seattle #seahawks last #nfl #pla...
70,#nfl #nflplayoffs #superbowl49 #superbowlchamp...
76,can't we just kick this thing off now? this #p...
107,@softstarshoes is giving away free moccasins i...
122,bam bam kam!! part of the #lob #nflplayoffs #s...
130,@softstarshoes is giving away free moccasins i...
207,you know what time it is. seahawks cake....\n\...


In [28]:
def create_stopwords():
    """stop_words_skt = text.ENGLISH_STOP_WORDS
    # nltk.download('stopwords')
    stop_words_en = stopwords.words('english')

    print("# of stop_words_skt:\t\t %s" % len(stop_words_skt))
    print("# of stop_words_en:\t\t %s" % len(stop_words_en))

    print("# of punctuation:\t\t %s" % len(punctuation))
    combined_stopwords = set.union(set(stop_words_en),set(punctuation),set(stop_words_skt))
    print("# of combined_stopwords:\t %s" % len(combined_stopwords))"""
    combined_stopwords=set(punctuation)
    return combined_stopwords

In [29]:

nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('punkt')
combined_stopwords = create_stopwords()
analyzer = CountVectorizer().build_analyzer()
wnl = nltk.wordnet.WordNetLemmatizer()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/shrutisharan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/shrutisharan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/shrutisharan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/shrutisharan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [30]:
count_vect = CountVectorizer(min_df=3,analyzer='word', stop_words= 'english')
X_counts = count_vect.fit_transform(dfs['text'])
print(X_counts.shape)
# print(count_vect.get_feature_names())


# Calculating the TF-IDF matrix for train and test data
tfidf_transformer = TfidfTransformer()
X_tfidf = tfidf_transformer.fit_transform(X_counts)
print(X_tfidf.shape)


(45576, 9516)
(45576, 9516)


In [31]:
from sklearn.decomposition import TruncatedSVD
def dimension_reduction_SVD(X_tfidf):
    svd = TruncatedSVD(n_components=1000)
    X_reduced = svd.fit_transform(X_tfidf)
    return X_reduced

In [32]:
X_reduced_svd = dimension_reduction_SVD(X_tfidf)

In [40]:
model= MLPClassifier()
clf=model.fit(X_tfidf,y_true)
y_pred=clf.predict(X_tfidf)


In [41]:
from sklearn.metrics import accuracy_score
accuracy_score(y_true,y_pred)

0.9887879585746885

In [35]:
model= LogisticRegression()
clf=model.fit(X_reduced_svd,y_true)
y_pred=clf.predict(X_reduced_svd)


/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [36]:
accuracy_score(y_true,y_pred)

0.7630551167281026

In [37]:
model= RandomForestClassifier()
clf=model.fit(X_reduced_svd,y_true)
y_pred=clf.predict(X_reduced_svd)


/usr/local/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [38]:
accuracy_score(y_true,y_pred)

0.9817228365806565